# Section 1: Social Security Trust Model

In [1]:
#import all the necessary packages
import numpy as np #for numerical array data
import pandas as pd #for tabular data
from scipy.optimize import minimize
import matplotlib.pyplot as plt #for plotting purposes
%matplotlib inline
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

import cvxpy as cp
from sklearn import datasets
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LassoCV
from sklearn.linear_model import Lasso
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

In [44]:
life_table = pd.read_csv('datasets/Life_expectancy.csv')
life_table.head()

,Age,Probability_M,Number_M,Life_M,Probability_F,Number_F,Life_F
0,0,0.006383,100000.0,76.15,0.005374,100000.0,80.97
1,1,0.000453,99362.0,75.63,0.000353,99463.0,80.41
2,2,0.000282,99317.0,74.67,0.000231,99427.0,79.44
3,3,0.000230,99289.0,73.69,0.000165,99405.0,78.45
4,4,0.000169,99266.0,72.71,0.000129,99388.0,77.47


In [242]:
# Parameters:
def SS_contribution(tax, payment, sex, inf_cum, int_cum, COLA_cum, n_people=100000, curr_age=25, death_reduce=.75/100):
    years_togo = 65 - curr_age
    yearly_contribution = np.zeros(len(inflation_cum))
    people_alive = n_people
    
    if sex == 'M':
        for i in range(years_togo):
            survive_prob = 1 - life_table['Probability_M'][int(curr_age+i)] * ((1-death_reduce)**i)
            people_alive = people_alive * survive_prob
            yearly_contribution[i] = people_alive * tax * inf_cum[i]
            
        for i in range(65,110):
            survive_prob = 1 - life_table['Probability_M'][int(i)] * ((1-death_reduce)**(i-curr_age))
            lumpsum_payment = 255 * (1-survive_prob) * people_alive
            people_alive = people_alive * survive_prob
            yearly_contribution[i-curr_age] = (-people_alive * payment - lumpsum_payment) * COLA_cum[i-curr_age]
            
        return yearly_contribution / int_cum
    
    else:
        for i in range(years_togo):
            survive_prob = 1 - life_table['Probability_F'][int(curr_age+i)] * ((1-death_reduce)**i)
            people_alive = people_alive * survive_prob
            yearly_contribution[i] = people_alive * tax * inf_cum[i]
            
        for i in range(65,110):
            survive_prob = 1 - life_table['Probability_F'][int(i)] * ((1-death_reduce)**(i-curr_age))
            lumpsum_payment = 255 * (1-survive_prob) * people_alive
            people_alive = people_alive * survive_prob
            yearly_contribution[i-curr_age] = (-people_alive * payment - lumpsum_payment) * COLA_cum[i-curr_age]
            
        return yearly_contribution / int_cum

In [232]:
discount_values = pd.DataFrame(np.concatenate((inflation_cum, COLA_cum)))

discount_values.to_excel('discount_values2.xlsx')

In [229]:
inflation_vec = 1.021 + np.random.randn(85) * 0.01  # Largest possible age: 110
COLA_vec = 0.1023 + 0.9 * inflation_vec + np.random.randn(85) * 0.009 # COLA...
interest_vec = inflation_vec + 0.003

inflation_cum = np.cumprod(inflation_vec)
interest_cum = np.cumprod(interest_vec)  # Discount rate
COLA_cum = np.cumprod(COLA_vec)

In [230]:
np.mean(inflation_vec), np.std(inflation_vec), np.mean(COLA_vec), np.std(COLA_vec)

(1.021423768314162,
 0.010389794146922102,
 1.0220560638426077,
 0.013456968484907322)

In [250]:
test_pay = 2861*12
test_income = 132900
test_rate = 13/100
test_tax = test_income * test_rate
print(np.sum(SS_contribution(test_tax, test_pay, 'M', inflation_cum, interest_cum, COLA_cum,n_people=53167) / 1e9))
print(np.sum(SS_contribution(test_tax, test_pay, 'F', inflation_cum, interest_cum, COLA_cum,n_people=46833) / 1e9))

3.148915786401107
-1.9274243091731404


# Section 2: DB Pension Fund Micro-Macro Model

In [251]:
all_data = pd.read_csv('MOOC/Data_Dec2018.csv')

factorname = ['World Equities','US Treasuries','High Yield ','Inflation Protection','Currency Protection']
Factors = all_data[factorname].values
assetname = ['World Equities','US Treasuries','High Yield ','US Equities','Real Estate','Commodities','Corp Bonds']
Assets = all_data[assetname].values
Tbill = all_data['T-bill'].values
Regime7 = all_data['Regime-7'].values.reshape(405)

n_time = Regime7.size
_, n_factor = Factors.shape
_, n_asset = Assets.shape

In [252]:
# EWMA covariance?
def ewma_cov(x1, x2, smoothing_factor = 0.96):
    # x1, x2 are two series
    
    n = len(x1)
    weights = np.array([smoothing_factor**(n-i-1) for i in range(n)])
    return np.sum(weights*x1*x2*(1-smoothing_factor)/(1-smoothing_factor**n))

Q_adj = np.zeros((n_asset,n_asset))

for i in range(n_asset):
    for j in range(n_asset):
        Q_adj[i,j] = ewma_cov(Assets[:,i],Assets[:,j])   

In [254]:
def regime_asset(n,mu1,mu2,Q1,Q2):
    s_1 = np.random.multivariate_normal(mu1, Q1, n).T
    s_2 = np.random.multivariate_normal(mu2, Q2, n).T
    regime = np.ones(n)
    for i in range(n-1):
        if regime[i] == 1:
            if np.random.rand() > 0.98:
                regime[i+1] = 0
        else:
            if np.random.rand() > 0.125:
                regime[i+1] = 0
    return (regime*s_1 + (1-regime)*s_2).T

In [261]:
# Scenarios
total_years = 85
mean_assets = np.array([[.52], [.27], [.47], [0.59], [0.45], [0.41], [0.35]]).reshape(7)
mean_normal = np.array([[1], [.15], [.8], [1.12], [0.84], [0.61], [0.33]]).reshape(7)
mean_crash = np.array([[-2.5], [1], [-1.6], [-2.7], [-1.79], [-0.8], [0.47]]).reshape(7)
Q_normal = np.cov(Assets[Regime7==1,:].T)
Q_crash = np.cov(Assets[Regime7==-1,:].T)
Q_adj = np.zeros((n_asset,n_asset))

for i in range(n_asset):
    for j in range(n_asset):
        Q_adj[i,j] = ewma_cov(Assets[:,i],Assets[:,j]) 

scenario_1 = []
for i in range(10000):
    scenario_1.append(np.random.multivariate_normal(mean_assets/100, Q_adj, total_years*12))

scenario_2 = []
for i in range(10000):
    scenario_2.append(regime_asset(total_years*12,mean_normal/100,mean_crash/100,Q_normal,Q_crash))

In [262]:
# Check simulated mean values
m1=np.zeros(7)
m2=np.zeros(7)
for i in range(10000):
    m1 += np.mean(scenario_1[i],axis=0) / 100
    m2 += np.mean(scenario_2[i],axis=0) / 100
    
m1, m2

(array([0.51915725, 0.27021174, 0.46966761, 0.5893056 , 0.44974933,
        0.41038246, 0.3502086 ]),
 array([0.51982226, 0.26636039, 0.4707132 , 0.59536603, 0.47932781,
        0.41879478, 0.3490333 ]))

In [263]:
def DB_contribution(contribution_rate, payment_rate, inf_cum, int_cum, scenario, portfolio_allocation,
                    n_people=100000, male_people=0.53167, curr_age=25, retire_age=65, death_reduce=.75/100):
    # Total "net" contribution to the fund
    # Assume initial salary = 1, 
    # scenario: asset returns, (year*number of assets) matrix
    # inf_cum: cumulative inflation, can also use wage path
    
    years_togo = retire_age - curr_age
    n_male = n_people * male_people
    n_female = n_people - n_male
    
    yearly_contribution = np.zeros(110-curr_age)
    yearly_investment = np.zeros(110-curr_age)
    yearly_payment = np.zeros(110-curr_age)
    total_asset = np.zeros(110-curr_age)
    
    for i in range(years_togo):
        n_female = n_female * (1 - life_table['Probability_F'][int(curr_age+i)] * ((1-death_reduce)**i))
        n_male = n_male * (1 - life_table['Probability_M'][int(curr_age+i)] * ((1-death_reduce)**i))
        people_alive = n_female + n_male
        yearly_contribution[i] = people_alive * contribution_rate * inf_cum[i]
        
        portfolio_return = np.cumprod(scenario[(12*i):(12*i+12),:].dot(portfolio_allocation) + 1)
        if i >= 1:
            yearly_investment[i] = total_asset[i-1] * (portfolio_return[-1]-1)
            total_asset[i] = total_asset[i-1] + yearly_investment[i] + yearly_contribution[i]
        else:
            total_asset[i] = yearly_contribution[i] # The first year, contribution only

    for i in range(years_togo,years_togo+110-retire_age):
        # Maximum age = 110
        n_female = n_female * (1 - life_table['Probability_F'][int(curr_age+i)] * ((1-death_reduce)**i))
        n_male = n_male * (1 - life_table['Probability_M'][int(curr_age+i)] * ((1-death_reduce)**i))
        people_alive = n_female + n_male
        yearly_payment[i] = people_alive * payment_rate * inf_cum[i]        
        
        portfolio_return = np.cumprod(scenario[(12*i):(12*i+12),:].dot(portfolio_allocation) + 1)
        
        #if total_asset[i-1] >= 0: Can add this for personal; for DB, investment continues no matter what
        yearly_investment[i] = total_asset[i-1] * (portfolio_return[-1]-1)
            
        total_asset[i] = total_asset[i-1] + yearly_investment[i] - yearly_payment[i]
    
    return np.sum(yearly_investment/int_cum), np.sum(yearly_contribution/int_cum), total_asset[-1]

In [296]:
sample_allocation = [0, 0.6, 0, 0.4, 0, 0, 0]
inflation_vec = 1.022 + np.random.randn(85) * 0.01  # Largest possible age: 110
interest_vec = inflation_vec + 0.01

inflation_cum = np.cumprod(inflation_vec)
interest_cum = np.cumprod(interest_vec)  # Discount rate

inflation_cum[-1] # Check: should be around 6.3

6.255564964893134

In [298]:
total_1 = np.zeros(10000)
total_2 = np.zeros(10000)

for i in range(10000):
    total_1[i] = DB_contribution(0.2, 0.8, inflation_cum, interest_cum, scenario_1[i], sample_allocation)[2]
    total_2[i] = DB_contribution(0.2, 0.8, inflation_cum, interest_cum, scenario_2[i], sample_allocation)[2]

In [299]:
np.sum(total_1>0), np.sum(total_2>0)

(8415, 6915)